<a href="https://colab.research.google.com/github/fitriindra/BigDataCourseExamples/blob/master/PySpark_DataProcessing_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Contoh pemrosesan data menggunakan RDD pada Dataset Titanic**

# **1. Instalasi Spark pada Google Colab**
kode di bawah ini hanya dijalankan **satu kali saja** saat runtime pertama kali dijalankan.

Sebelum menjalankan, uncomment baris-baris kode di bawah ini

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# **2. Import library Spark yang sudah diinstal**

In [3]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import col, avg

# **3. Inisialisasi SparkContext dan SparkSession**

In [4]:
sc = spark.sparkContext
spark = SparkSession(sc)

# **4. Load Dataset**

Pada langkah ini, dataset berupa file CSV di-load. Dataset merupakan data yang sudah dilakukan proses pre-processing (lihat contoh [```PySpark EDA Example```](https://colab.research.google.com/drive/1NQth__UYjDBOG-b-RsvY56siYdGGn8r0?usp=sharing)).
Lokasi file csv sama dengan lokasi file kode program

In [5]:
titanic_df = spark.read.csv("titanic_processed.csv", header=True, inferSchema=True)
titanic_df.printSchema()            #mengecek schema dari dataset

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund Mr. Owen H...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings Mrs. John...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen Miss. L...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle Mrs. Jac...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen Mr. William...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

# **5. Pemrosesan Data**

## 5.1 Pemrosesan Menggunakan Transformasi dan Action pada DataFrame

Contoh pemrosesan:

Menghitung rata-rata umur penumpang yang selamat berdasarkan jenis kelamin.

Langkahnya adalah:
1.   Melakukan filter terhadap kolom Survived. Data dengan nilai ```Survived = 1``` berarti penumpang selamat. Data dengan filter ```Survived = 1``` inilah yang akan dipakai pada proses berikutnya
2.   Melakukan perhitungan rata-rata umur. Perhitungan dilakukan dengan melakukan grouping berdasarkan jenis kelamin kemudian dihitung rata-rata per jenis kelamin. Proses ini dapat digabung menjadi satu baris kode (lihat di bawah)


In [7]:
#filter kolom Survived yang nilainya 1 (selamat)
titanic_survived = titanic_df.filter(titanic_df.Survived == 1)
#jumlah total penumpang selamat
print("jumlah penumpang selamat:", titanic_survived.count())
#perhitungan rata-rata penumpang berdasarkan jenis kelamin
titanic_survived.groupby("Sex").avg("Age").show()

jumlah penumpang selamat: 342
+------+------------------+
|   Sex|          avg(Age)|
+------+------------------+
|female|28.979262812421116|
|  male| 27.63170534268753|
+------+------------------+



## 5.2 Pemrosesan Menggunakan SparkSQL

Contoh pemrosesan:

Contoh studi kasus "Menghitung rata-rata umur penumpang yang selamat berdasarkan jenis kelamin" dapat dikerjakan menggunakan beberapa cara, menggunakan fungsi transformations dan actions pada RDD yang telah dikerjakan di 5.1 atau menggunakan DataFrame dan SparkSQL.


In [8]:
#membuat table (view)
titanic_survived.createOrReplaceTempView("titanic")

#menggunakan PySparkSQL untuk mencari rata-rata penumpang selamat berdasarkan jenis kelamin
df_result = spark.sql("SELECT Sex, AVG(Age) FROM titanic WHERE Survived = 1 GROUP BY Sex")
df_result.show()

+------+------------------+
|   Sex|          avg(Age)|
+------+------------------+
|female|28.979262812421116|
|  male| 27.63170534268753|
+------+------------------+



hasil menggunakan transformations dan actions pada langkah 5.1 sama dengan hasil menggunakan PySparkSQL pada langkah 5.2

In [9]:
spark.stop()    #digunakan untuk menghentikan SparkSession